### Geno Encoding

In [23]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, output_kernel_size, padding):
        super(Encoder, self).__init__()
        
        self.conv_layer = nn.Conv2d(in_channels=input_size[0],
                                    out_channels=output_kernel_size[0],
                                    kernel_size=output_kernel_size[1:],
                                    padding=padding)
        
        self.activation = nn.ReLU()
        
    def forward(self, x):
        x = self.conv_layer(x)
        x = self.activation(x)
        return x

def create_encoder(input_size, output_kernel_size, padding):
    return Encoder(input_size, output_kernel_size, padding)

def mutate_encoder(encoder, mutation_rate=0.1):
    mutated_encoder = Encoder(
        input_size=encoder.conv_layer.in_channels,
        output_kernel_size=encoder.conv_layer.out_channels,
        padding=encoder.conv_layer.padding[0]
    )
    
    for param, mutated_param in zip(encoder.parameters(), mutated_encoder.parameters()):
        if torch.rand(1) < mutation_rate:
            mutated_param.data = param.data + torch.randn_like(param.data) * 0.1  # Adding noise
    
    return mutated_encoder

# Example usage
input_size = (3, 64, 64)  # Assuming RGB images of size 64x64
output_kernel_size = (16, 3, 3)  # 16 output channels, 3x3 kernel
padding = 1
encoder = create_encoder(input_size, output_kernel_size, padding)
print(encoder)

mutated_encoder = mutate_encoder(encoder)
print(mutated_encoder)


Encoder(
  (conv_layer): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (activation): ReLU()
)


TypeError: 'int' object is not subscriptable

In [27]:
import torch.nn as nn

class EncodedNetwork(nn.Module):
    def __init__(self, architecture):
        super(EncodedNetwork, self).__init__()
        
        layers = architecture.split('-')
        
        layer_units = []
        for layer in layers :
                if layer.startswith('I'):
                    self.input_units = int(layer[1:])
                elif layer.startswith('O'):
                    self.output_units = int(layer[1:])
                elif layer.startswith("P"):
                    self.padding=int(layer[1:])
                elif layer.startswith("k"):
                    self.kernel=int(layer[1:])
                elif layer.startswithe("S"):
                    self.stride=int(layer[1:])
                    
            
        self.layers = []
        for units in 10:
                self.layers.append(nn.Conv2d(self.input_units, self.output_units,kernel_size=(self.kernel,self.kernel),stride=(self.stride,self.stride),padding=(self.padding,self.padding)))
                self.layers.append(nn.ReLU())
                self.input_units = units
            
        self.layers.append(nn.Linear(self.input_units, self.output_units))
            
        self.network = nn.Sequential(*self.layers)
    
    def forward(self, x):
        return self.network(x)

# Example encoded architecture



In [28]:
population="I1,016,P2,K2,S1-I16,032,P2,K2,S1"
net=EncodedNetwork(population)

ValueError: invalid literal for int() with base 10: '1,016,P2,K2,S1'

In [29]:
import torch.nn as nn

class Encodingnetwork(nn.Module):
    def __init__(self, architecture):
        super(Encodingnetwork, self).__init__()
        
        layers = architecture.split('-')
        
        layer_units = []
        for l in layers:
            cells=l.split("_")               
            for layer in cells :
                if layer.startswith('I'):
                    self.input_units = int(layer[1:])
                elif layer.startswith('O'):
                    self.output_units = int(layer[1:])
                elif layer.startswith("P"):
                    self.padding=int(layer[1:])
                elif layer.startswith("k"):
                    self.kernel=int(layer[1:])
                elif layer.startswithe("S"):
                    self.stride=int(layer[1:])
                elif layer.startswith("Inception"):
                    self.inception_module = nn.ModuleList([
                        nn.Conv2d(self.input_units, 64, kernel_size=(1,1), stride=(1,1), padding=(0,0)),
                        nn.Conv2d(self.input_units, 128, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                        nn.Conv2d(self.input_units, 256, kernel_size=(5,5), stride=(1,1), padding=(2,2)),
                    ])
                    self.output_units = 256
                    
            
            self.layers = []
            for units in layer_units:
                if layer.startswith("Inception"):
                    self.layers.append(self.inception_module[0])
                    self.layers.append(nn.ReLU())
                    self.layers.append(self.inception_module[1])                    
                    self.layers.append(nn.ReLU())
                    self.layers.append(self.inception_module[2])
                    self.layers.append(nn.ReLU())
                else:
                    self.layers.append(nn.Conv2d(self.input_units, self.output_units,kernel_size=(self.kernel,self.kernel),stride=(self.stride,self.stride),padding=(self.padding,self.padding)))
                    self.layers.append(nn.ReLU())
                    self.input_units = units
            
            self.layers.append(nn.Linear(self.input_units, self.output_units))
            
            self.network = nn.Sequential(*self.layers)
    
    def forward(self, x):
        return self.network(x)

# Example encoded architecture



In [5]:
import torch
import torch.nn as nn

# Define an architecture string
architecture = "I3-Conv2_32-Conv2_64-Inception-ResNet-Conv2_128-Conv2_256-FC_10"

# Create an instance of EncodedNetwork
population = 64  # You can set the population value here
net = Encodingnetwork(architecture)

# Create a random input tensor for testing
batch_size = 16
channels = 3
input_size = 64
x = torch.randn(batch_size, channels, input_size, input_size)

# Forward pass through the network
output = net(x)

# Print the shape of the output tensor
print("Output shape:", output.shape)


AttributeError: 'Encodingnetwork' object has no attribute 'output_units'

In [ ]:
Population=["I1_O16_P2_S1_K5-I16_O32_P2_S1_K5-I32_O64_P2_S1_K5-I64_O128_P2_S1_K5"]
Population      

In [ ]:
net=Encodingnetwork(Population)

In [ ]:
"badar".startswith("ba")

In [ ]:
network

In [ ]:
#function to minimize
def fitness_function(network, dataloader):
    network.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in dataloader:
            outputs = network(images)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return correct / total

In [ ]:
#downloading the dataset
from torchvision import datasets, transforms
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
train_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
test_dataset = datasets.MNIST('../data', train=False,
                   transform=transform)


In [ ]:
import torch
from torch import nn
from evotorch.neuroevolution.net import count_parameters

class MNIST30K(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        # The first convolution uses a 5x5 kernel and has 16 filters
        self.conv1 = nn.Conv2d(1, 16, kernel_size = 5, stride = 1, padding=2)
        # Then max pooling is applied with a kernel size of 2
        self.pool1 = nn.MaxPool2d(kernel_size = 2)
        # The second convolution uses a 5x5 kernel and has 32 filters
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 5, stride = 1, padding = 2)
        # Another max pooling is applied with a kernel size of 2
        self.pool2 = nn.MaxPool2d(kernel_size = 2)

        # The authors are unclear about when they apply batchnorm. 
        # As a result, we will apply after the second pool
        self.norm = nn.BatchNorm1d(1568, affine = False)

        # A final linear layer maps outputs to the 10 target classes
        self.out = nn.Linear(1568, 10)

        # All activations are ReLU
        self.act = nn.ReLU()

    def forward(self, data: torch.Tensor) -> torch.Tensor:
        # Apply the first conv + pool
        data = self.pool1(self.act(self.conv1(data)))
        # Apply the second conv + pool
        data = self.pool2(self.act(self.conv2(data)))

        # Apply layer norm
        data = self.norm(data.flatten(start_dim = 1))

        # Flatten and apply the output linear layer
        data = self.out(data)

        return data

network = MNIST30K()
print(f'Network has {count_parameters(network)} parameters')

In [ ]:
network

In [ ]:
from torchvision import datasets, transforms

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
train_dataset = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
test_dataset = datasets.MNIST('../data', train=False,
                   transform=transform)

In [ ]:
MNIST30K

In [ ]:
from evotorch.neuroevolution import SupervisedNE

mnist_problem = SupervisedNE(
    train_dataset,  # Using the dataset specified earlier
    EncodedNetwork(population[0]),  # Training the MNIST30K module designed earlier
    nn.CrossEntropyLoss(),  # Minimizing CrossEntropyLoss
    minibatch_size = 256,  # With a minibatch size of 256
    common_minibatch = True,  # Always using the same minibatch across all solutions on an actor
    num_actors = 4,  # The total number of CPUs used
    num_gpus_per_actor = 'max',  # Dividing all available GPUs between the 4 actors
    subbatch_size = 50,  # Evaluating solutions in sub-batches of size 50 ensures we won't run out of GPU memory for individual workers
)

In [ ]:
from evotorch.algorithms import SNES
searcher = SNES(mnist_problem, stdev_init = 1, popsize = 1000, distributed = True)

In [ ]:
from evotorch.logging import StdOutLogger, PandasLogger
stdout_logger = StdOutLogger(searcher, interval = 1)
pandas_logger = PandasLogger(searcher, interval = 1)

In [ ]:
searcher.run(10)

In [ ]:
pandas_logger.to_dataframe().mean_eval.plot()

In [ ]:
net = mnist_problem.parameterize_net(searcher.status['center']).cpu()

loss = torch.nn.CrossEntropyLoss()
net.eval()
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 256, shuffle = False)
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        output = net(data)
        test_loss += loss(output, target).item() * data.shape[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
print('Test set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
test_loss, correct, len(test_loader.dataset),
100. * correct / len(test_loader.dataset)))

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [20]:
def create_neural_networks(population):
  """Creates a list of neural networks from the given population.

  Args:
    population: The population of the neural networks, in the format
      "I<number of inputs>,P<number of hidden layers>,K<number of neurons per hidden layer>,S<number of outputs>-I<number of inputs>,P<number of hidden layers>,K<number of neurons per hidden layer>,S<number of outputs>".

  Returns:
    A list of neural networks.
  """

  # Split the population into its components.
  neural_networks = []
  for network in population.split("-"):
    neural_networks.append(create_neural_networks(network))

  # Return the neural networks.
  return neural_networks


In [21]:
import numpy as np

class NeuralNetworks:

  def __init__(self, number_of_inputs, number_of_hidden_layers, number_of_neurons_per_hidden_layer, number_of_outputs):
    self.number_of_inputs = number_of_inputs
    self.number_of_hidden_layers = number_of_hidden_layers
    self.number_of_neurons_per_hidden_layer = number_of_neurons_per_hidden_layer
    self.number_of_outputs = number_of_outputs
    self.weights = np.random.rand(number_of_inputs, number_of_neurons_per_hidden_layer)
    self.biases = np.random.rand(number_of_neurons_per_hidden_layer)

  def forward_propagate(self, inputs):
    """Propagates the input data through the neural network and returns the output."""

    layer_outputs = []
    for i in range(self.number_of_hidden_layers):
      layer_outputs.append(np.dot(inputs, self.weights[i]) + self.biases[i])
    inputs = layer_outputs[-1]

    return inputs

  def backward_propagate(self, errors):
    """Propagates the error through the neural network and updates the weights and biases."""

    for i in reversed(range(self.number_of_hidden_layers)):
      errors = np.dot(errors, self.weights[i].T)
      self.weights[i] += np.dot(inputs.T, errors)
      self.biases[i] += errors

  def predict(self, inputs):
    """Takes in an input and returns the predicted output."""

    outputs = self.forward_propagate(inputs)
    return outputs

  def train(self, dataset):
    """Trains the neural network on a dataset."""

    for inputs, outputs in dataset:
      errors = outputs - self.predict(inputs)
      self.backward_propagate(errors)



In [22]:
population = "I1,016,P2,K2,S1-I16,032,P2,K2,S1"
neural_networks = create_neural_networks(population)


RecursionError: maximum recursion depth exceeded